## shNfkb1-lane1

2/2/24 - Data generated from shNfkb1 single hairpin experiments; included shRNA hairpin library, nonbarcoded; data submitted by Jason/Hannah/CHP on 5/23/2023

Analyze global run on cellranger v6 data and create h5_ad files for further analysis

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc

import math
import matplotlib

#from solo import hashsolo
import anndata

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')
np.random.seed(1573)   #fix so we can reproduce later

In [ ]:
results_file = 'write/2434-nfkb1.h5ad'  # the file that will store the analysis results

Read in the count matrix into an [`AnnData`](https://anndata.readthedocs.io/en/latest/anndata.AnnData.html) object, which holds many slots for annotations and different representations of the data. It also comes with its own HDF5 file format: `.h5ad`.

In [ ]:
adata = sc.read_10x_h5('filtered_feature_bc_matrix.h5', gex_only=False)

In [ ]:
adata.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`

In [ ]:
samples = list(adata[:,adata.var['feature_types']=='Antibody Capture'].var.index)

In [ ]:
print(samples)

In [ ]:
# Now filter out barcodes
hashadata = adata[:,samples]
adata = adata[:,[y for y in adata.var_names if y not in samples]]

In [ ]:
hashadata.var

## Preprocessing

Show those genes that yield the highest fraction of counts in each single cell, across all cells.

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )

Basic filtering:

In [ ]:
sc.pp.filter_cells(adata, min_counts=1500)
sc.pp.filter_cells(adata, min_genes=300)
#sc.pp.filter_genes(adata, min_cells=0.1*len(adata.obs))

Let's assemble some information about mitochondrial genes, which are important for quality control.

Citing from "Simple Single Cell" workflows [(Lun, McCarthy & Marioni, 2017)](https://master.bioconductor.org/packages/release/workflows/html/simpleSingleCell.html#examining-gene-level-metrics):

> High proportions are indicative of poor-quality cells (Islam et al. 2014; Ilicic et al. 2016), possibly because of loss of cytoplasmic RNA from perforated cells. The reasoning is that mitochondria are larger than individual transcript molecules and less likely to escape through tears in the cell membrane.

With `pp.calculate_qc_metrics`, we can compute many metrics very efficiently.

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('mt-') # annotate the group of mitochondrial genes as 'MT'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

A violin plot of some of the computed quality measures:

* the number of genes expressed in the count matrix
* the total counts per cell
* the percentage of counts in mitochondrial genes

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True, rotation=90)

In [ ]:
adata.obs["log10GenesPerUMI"] = adata.obs['n_genes_by_counts'].apply(math.log10) / adata.obs['total_counts'].apply(math.log10)
matplotlib.pyplot.hist(adata.obs["log10GenesPerUMI"])

In [ ]:
sc.pl.violin(adata, ['log10GenesPerUMI'],
             jitter=0.4, multi_panel=True, rotation=90)
#99% seems like a good Human pct count cutoff

Keep singlets, remove cells that have too many mouse reads, or mitochondrial genes expressed or too many total counts:

In [ ]:
#Filter by Log10GenesPerUMI
adata = adata[adata.obs.log10GenesPerUMI >= 0.8,:]

In [ ]:
#adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt <= 10, :]

## Run Demultiplexing

In [ ]:
hashadata = hashadata[hashadata.obs.index.isin(adata.obs.index), :]

In [ ]:
hashadata.var_names

In [ ]:
hashCounts = pd.DataFrame(hashadata.X.todense(), columns=hashadata.var_names, index=adata.obs.index)

In [ ]:
hashDisc = hashCounts.describe([.1,.2,.3,.4,.5,.6,.7,.8,.9,0.99])
hashDisc

In [ ]:
hashIDs = hashCounts.copy()
hashID = hashadata.var_names
for hashName in hashadata.var_names:
    print(hashName)
    print(hashDisc.loc["90%",hashName])
    hashIDs.loc[:,hashName] = hashCounts.loc[:,hashName] > hashDisc.loc["90%",hashName]
hashIDs

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
hashCounts

In [ ]:
plt.rcParams['figure.figsize'] = (4,4)

In [ ]:
fig, axs = plt.subplots(2,5, figsize =(20, 5))

for i, hashName in enumerate(hashadata.var_names):
    hashCounts2 = np.log10(hashCounts[hashName]+1)
    axs[i//5,i%5].set_title(hashName)
    axs[i//5,i%5].hist(hashCounts2, bins = 100)
    axs[i//5,i%5].axvline(np.log10(int(hashDisc.loc["90%",hashName])+1), color='k', linestyle='dashed', linewidth=1)

#plt.rcParams["figure.figsize"] = (20,5)
plt.show()

In [ ]:
hashName= "AW1781_Renilla"
hash301 = np.log10(hashCounts[hashName]+1)

fig, ax = plt.subplots(figsize =(10, 7))
ax.hist(hash301, bins = 100)
ax.axvline(np.log10(int(hashDisc.loc["90%",hashName])+1), color='k', linestyle='dashed', linewidth=1)
# Show plot
plt.show()

In [ ]:
from sklearn.mixture import BayesianGaussianMixture
from scipy.stats import norm
fig, axs = plt.subplots(2,5, figsize =(20, 5))
dfHashBoundry = pd.DataFrame(np.zeros(len(hashadata.var_names)),hashadata.var_names, columns=["boundry"])
gmm = BayesianGaussianMixture(n_components=2, random_state=250,init_params='k-means++')
binEx = np.arange(0,5,5/100).reshape(-1,1)

for i, hashName in enumerate(hashadata.var_names):
    hashCount = np.array(np.log10(hashCounts[hashName]+1)).reshape(-1, 1)
    fitGMM = gmm.fit(hashCount)
    mean = fitGMM.means_  
    covs  = fitGMM.covariances_
    weights = fitGMM.weights_
  
    fitGmmBound = fitGMM.predict(binEx)

    x_axis = np.arange(0, 5, 0.1)
    y_axis0 = norm.pdf(x_axis, float(mean[0][0]), np.sqrt(float(covs[0][0][0])))*weights[0] # 1st gaussian
    y_axis1 = norm.pdf(x_axis, float(mean[1][0]), np.sqrt(float(covs[1][0][0])))*weights[1] # 2nd gaussian

    hashBoundry = False #binEx[np.where(fitGmmBound == 1)[0][0]][0]    
    if mean[0][0] < mean[1][0]:
        hashBoundry = x_axis[np.where(y_axis1 < y_axis0)[0][-1]+2]
    else:
        hashBoundry = x_axis[np.where(y_axis0 < y_axis1)[0][-1]+2]
    
    naiveBoundry = np.log10(int(hashDisc.loc["90%",hashName])+1)
    
    dfHashBoundry.loc[hashName] = hashBoundry
    
    # Plot 2
    axs[i//5,i%5].set_title(hashName)
    axs[i//5,i%5].axvline(naiveBoundry, c='C3', linestyle='dashed', linewidth=1) #red
    axs[i//5,i%5].axvline(hashBoundry, c='C2', linestyle='dashed', linewidth=1)  #green
    axs[i//5,i%5].hist(hashCount, density=True, color='black', bins=100)        
    axs[i//5,i%5].plot(x_axis, y_axis0, lw=3, c='C6')                            #pink
    axs[i//5,i%5].plot(x_axis, y_axis1, lw=3, c='C1')                            #orange
    axs[i//5,i%5].plot(x_axis, y_axis0+y_axis1, lw=3, c='C0', ls=':')            #dotted blue
    
plt.tight_layout(pad=1.0)
#plt.rcParams["figure.figsize"] = (20,5)
plt.show()

In [ ]:
hashIDs = hashCounts.copy()
hashID = hashadata.var_names
for hashName in hashadata.var_names:
    print(hashName)
    print(dfHashBoundry.loc[hashName].values[0])
    hashIDs.loc[:,hashName] = np.log10(hashCounts.loc[:,hashName]+1) > dfHashBoundry.loc[hashName].values[0]
#hashIDs.loc[:,'AT1129_B0304_2117'] = False
#hashIDs.loc[:,'AT1606_B0305_Renilla'] = False
#hashIDs.loc[:,'AT1831_B0306_2117'] = False
#hashIDs.loc[:,'AT1607_B0307_Renilla'] = False
#hashIDs.loc[:,'AV1189_B0310_2118'] = False
hashIDs

In [ ]:
classification = np.empty(len(adata), dtype="object")
i = 0
for cellBar, hashBool in hashIDs.iterrows():
    #print(i)
    #print(hashBool)
    #print(hashBool.values)
    #print(sum(hashBool))
    numHashes = sum(hashBool)
    if (numHashes == 1):
        classif = hashID[hashBool.values].values[0]
    elif (numHashes > 1):
        classif = "doublet"
    else:
        classif = "negative"
    classification[i] = classif
    i = i + 1
    #break

In [ ]:
adata.obs["Classification"] = classification
hashadata.obs["Classification"] = classification
adata.obs["Classification"].value_counts()

In [ ]:
#output visulaization of hashing
sc.pl.heatmap(hashadata, hashadata.var_names, groupby="Classification", log=True)#, save = f"_{figName}_hash.png")

## Scrublet

In [ ]:
import scrublet as scr
import scipy.io

In [ ]:
doublet_calls = np.array(adata.obs.Classification) == 'doublet'
naive_doublet_rate = doublet_calls.sum() / float(len(doublet_calls))

In [ ]:
expected_doublet_rate = naive_doublet_rate * len(samples) / (len(samples)-1)
print("Expected doublet rate: %0.2f" % expected_doublet_rate)

In [ ]:
scrub = scr.Scrublet(adata.X, expected_doublet_rate=0.128)
#scrub = scr.Scrublet(adata.X, expected_doublet_rate=expected_doublet_rate)

In [ ]:
doublet_scores, predicted_doublets = scrub.scrub_doublets(n_prin_comps=30, 
                                                            mean_center=True, 
                                                            normalize_variance=True)

In [ ]:
#scrub.call_doublets(threshold=0.3)

In [ ]:
scrub.plot_histogram();

In [ ]:
print('Running UMAP...')
scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))

# # Uncomment to run tSNE - slow
# print('Running tSNE...')
# scrub.set_embedding('tSNE', scr.get_tsne(scrub.manifold_obs_, angle=0.9))

# # Uncomment to run force layout - slow
# print('Running ForceAtlas2...')
# scrub.set_embedding('FA', scr.get_force_layout(scrub.manifold_obs_, n_neighbors=5. n_iter=1000))
    
print('Done.')

In [ ]:
scrub.plot_embedding('UMAP', order_points=True);

# scrub.plot_embedding('tSNE', order_points=True);
# scrub.plot_embedding('FA', order_points=True);

In [ ]:
doub_calls = {'Doublet': doublet_calls,
              'Scrublet (z-score)': scrub.predicted_doublets_, 
              }

In [ ]:
x = scrub._embeddings['UMAP'][:,0]
y = scrub._embeddings['UMAP'][:,1]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (14, 5))
for iMethod, method in enumerate(doub_calls):
    coldat = doub_calls[method]
    o = np.argsort(coldat)

    ax = axs[iMethod]
    ax.scatter(x[o], y[o], c = coldat[o], cmap=scr.custom_cmap([[.7,.7,.7], [0,0,0]]), s = 2)
    ax.set_xticks([]), ax.set_yticks([])
    ax.set_title(method)
fig.tight_layout()

In [ ]:
adata.obs['Scrublet_doublet']=scrub.predicted_doublets_

In [ ]:
 cell_proportion_df = pd.crosstab(adata.obs['Classification'],adata.obs['Scrublet_doublet'], normalize='columns').T.plot(kind='bar', stacked=True).legend(bbox_to_anchor=(1,1))

## Examine raw data

In [ ]:
hadata = adata.copy()

In [ ]:
sc.pp.normalize_total(hadata, target_sum=1e4)

Logarithmize the data:

In [ ]:
sc.pp.log1p(hadata)

In [ ]:
sc.pp.highly_variable_genes(hadata)

In [ ]:
hadata = hadata[:, hadata.var.highly_variable]

In [ ]:
sc.pp.scale(hadata, max_value=10)

## Principal component analysis

Reduce the dimensionality of the data by running principal component analysis (PCA), which reveals the main axes of variation and denoises the data.

In [ ]:
sc.tl.pca(hadata, svd_solver='arpack', n_comps=150)

We can make a scatter plot in the PCA coordinates, but we will not use that later on.

Let us inspect the contribution of single PCs to the total variance in the data. This gives us information about how many PCs we should consider in order to compute the neighborhood relations of cells, e.g. used in the clustering function  `sc.tl.louvain()` or tSNE `sc.tl.tsne()`. In our experience, often a rough estimate of the number of PCs does fine.

In [ ]:
sc.pl.pca_variance_ratio(hadata, log=True, n_pcs=150)

## Computing the neighborhood graph

Let us compute the neighborhood graph of cells using the PCA representation of the data matrix. You might simply use default values here. For the sake of reproducing Seurat's results, let's take the following values.

In [ ]:
sc.pp.neighbors(hadata, n_neighbors=15, n_pcs=75)

## Embedding the neighborhood graph

We suggest embedding the graph in two dimensions using UMAP ([McInnes et al., 2018](https://arxiv.org/abs/1802.03426)), see below. It is potentially more faithful to the global connectivity of the manifold than tSNE, i.e., it better preserves trajectories. In some ocassions, you might still observe disconnected clusters and similar connectivity violations. They can usually be remedied by running:

```
tl.paga(adata)
pl.paga(adata, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
tl.umap(adata, init_pos='paga')
```

In [ ]:
sc.tl.umap(hadata)

In [ ]:
sc.pl.umap(hadata, color='Classification', groups=['doublet','negative'])
#for label in hadata.obs['Classification'].unique():
#    sc.pl.umap(hadata, color='Classification',groups=[label])

In [ ]:
sc.pl.umap(hadata, color='Classification')

In [ ]:
samples

In [ ]:
sc.pl.umap(hadata, color='Classification', groups=samples)

In [ ]:
sc.pl.violin(hadata,['2117-Nfkb1.2331','2118-Nfkb1.3737'], groupby='Classification', rotation=90)

In [ ]:
hadata.obs.Classification.value_counts()

## set highly variable genes before moving all mean to 0

In [ ]:
#Remove any cell with PTPRC counts
fig, ax = plt.subplots(figsize =(10, 7))
ax.hist(adata.X[:,adata.var.index.isin(['shRenilla'])].toarray())
#ax.axvline(np.log10(int(hashDisc.loc["90%",hashName])+1), color='k', linestyle='dashed', linewidth=1)
# Show plot
#adata.obs['shRen'] = 0

#adata.obs['shRenilla'][adata.X[:,adata.var.index.isin(['shRenilla'])] >0] #= adata.X[:,adata.var.index.isin(['shRenilla'])]

In [ ]:
adata.obs['shRen'] = adata.X[:,adata.var.index.isin(['shRenilla'])].toarray()

In [ ]:
#Remove any cell with PTPRC counts
fig, ax = plt.subplots(figsize =(10, 7))
ax.hist(adata.X[:,adata.var.index.isin(['2118-Nfkb1.3737'])].toarray())
#ax.axvline(np.log10(int(hashDisc.loc["90%",hashName])+1), color='k', linestyle='dashed', linewidth=1)
# Show plot
#adata.obs['2118-Nfkb1.3737'] = 0

#adata.obs['shRenilla'][adata.X[:,adata.var.index.isin(['shRenilla'])] >0] #= adata.X[:,adata.var.index.isin(['shRenilla'])]

In [ ]:
adata.obs['2118-Nfkb1'] = adata.X[:,adata.var.index.isin(['2118-Nfkb1.3737'])].toarray()

In [ ]:
#Remove any cell with PTPRC counts
fig, ax = plt.subplots(figsize =(10, 7))
ax.hist(adata.X[:,adata.var.index.isin(['2117-Nfkb1.2331'])].toarray())
#ax.axvline(np.log10(int(hashDisc.loc["90%",hashName])+1), color='k', linestyle='dashed', linewidth=1)
# Show plot
#adata.obs['2117-Nfkb1.2331'] = 0

#adata.obs['shRenilla'][adata.X[:,adata.var.index.isin(['shRenilla'])] >0] #= adata.X[:,adata.var.index.isin(['shRenilla'])]

In [ ]:
adata.obs['2117-Nfkb1'] = adata.X[:,adata.var.index.isin(['2117-Nfkb1.2331'])].toarray()

In [ ]:
import matplotlib.pyplot as plt
adata.var.index.isin(['GFP'])
_ = plt.hist(adata[adata.obs.index,'GFP'].X.toarray())
plt.ylabel("Count")
plt.title("GFP")
plt.axvline(x=2)
plt.show()

In [ ]:
adata.obs['GFP_count'] = adata.X[:,adata.var.index.isin(['GFP'])].toarray()

In [ ]:
import matplotlib.pyplot as plt
adata.var.index.isin(['GFP'])
_ = plt.hist(adata[adata.obs.index,'Cre'].X.toarray())
plt.ylabel("Count")
plt.title("Cre")
plt.axvline(x=2)
plt.show()

In [ ]:
adata.obs['Cre_count'] = adata.X[:,adata.var.index.isin(['Cre'])].toarray()

## Filter doublets and negative cells

In [ ]:
adata.obs.Classification.value_counts()

In [ ]:
adata = adata[~(adata.obs['Classification'].isin(['doublet','negative']) | adata.obs.Scrublet_doublet),:]

In [ ]:
adata.obs.Classification.value_counts()

## Filter genes ( >= 1% of cells )

In [ ]:
sc.pp.filter_genes(adata, min_cells=0.01*len(adata.obs))

## Set Groups

In [ ]:
#Set shRNA groups
adata.obs['shRNA'] = None
adata.obs['shRNA'][adata[adata.obs['shRen'] >= 2,:].obs.index] = 'shRenilla'
adata.obs['shRNA'][adata[adata.obs['2117-Nfkb1'] >= 2,:].obs.index] = '2117'
adata.obs['shRNA'][adata[adata.obs['2118-Nfkb1'] >= 2,:].obs.index] = '2118'

In [ ]:
#Set shRNA groups
adata.obs['Group'] = 'Renilla_Control'
adata.obs['Group'][adata.obs.Classification.isin(['AW1781_Renilla','AX1068_Renilla'])] = 'Renilla_EGFRi'
adata.obs['Group'][adata.obs.Classification.isin(['AX1127_2117','AX1893_2117'])] = '2117_EGFRi'
adata.obs['Group'][adata.obs.Classification.isin(['AW1785_2118'])] = '2118_EGFRi'
adata.obs['Group'][adata.obs.Classification.isin(['AX1508_2117','AX1598_2117'])] = '2117_Control'
adata.obs['Group'][adata.obs.Classification.isin(['AW1555_2118'])] = '2118_Control'

## Double check shRNAs and filter accordingly

In [ ]:
sc.pl.violin(adata,['shRen','2117-Nfkb1','2118-Nfkb1'], groupby='Classification', rotation=90)

In [ ]:
adata.obs[['shRNA','Group']].value_counts()

In [ ]:
adata = adata[~((adata.obs.shRNA.isin(['2117','shRenilla']) & adata.obs.Group.isin(['2118_EGFRi'])) | 
                (adata.obs.shRNA.isin(['shRenilla']) & adata.obs.Group.isin(['2117_EGFRi','2117_Control']))),:]

In [ ]:
adata.obs[['shRNA','Group']].value_counts()

In [ ]:
sc.pl.violin(adata,['shRen','2117-Nfkb1','2118-Nfkb1'], groupby='Group', rotation=90)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True, rotation=90)

In [ ]:
adata.layers['counts']=adata.X.copy()

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

Logarithmize the data:

In [ ]:
sc.pp.log1p(adata)

In [ ]:
sc.pp.highly_variable_genes(adata) #batch_key=Classification

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
library = ["1659-Fosl2.226","1660-Fosl2.752","1661-Fhl2.467","1662-Fhl2.179","1663-Myc.989","1664-Myc.1389","1665-Yap1.735","1666-Yap1.2847","1667-Atf4.1383","1668-Atf4.536","1669-Maff.85","1670-Maff.587","1671-Nfkb2.152","1672-Nfkb2.534","1673-Relb.286","1674-Relb.1795","1675-Onecut2.1799","1676-Onecut2.2361","1810-Slc4a11.541","1934-Slc4a11.1966","1811-Itga2.672","1812-Itga2.204","1998-Cldn4-1013","1999-Cldn4-951","2113-Lif.440","2114-Lif.1890","2115-Fosl1.1401","2116-Fosl1.699","2117-Nfkb1.2331","2118-Nfkb1.3737","2190-Jun_2481","2191-Jun_3136","2193-JunB_406","2194-JunD_78","2195-JunD_859","JCP618-Rela_1202","JCP621-Rela_1182","JCP623-Rel_1868","JCP625-Rel_342","JCP628-Ikbkg_2526","JCP629-Ikbkg_2984"]

In [ ]:
#Ensure marker genes aren't making an impact to highly variable gene analysis downstream
adata.var.highly_variable['GFP'] = False
adata.var.highly_variable['mKate2'] = False
adata.var.highly_variable['Cre'] = False
for x in library:
    adata.var.highly_variable[x] = False

In [ ]:
adata.raw = adata

In [ ]:
adata.write('write/allmice.h5ad', compression='gzip')

In [ ]:
adata = sc.read('write/allmice.h5ad')

In [ ]:
adata = adata[:, adata.var.highly_variable]

In [ ]:
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

In [ ]:
sc.pp.scale(adata, max_value=10)

## Principal component analysis

Reduce the dimensionality of the data by running principal component analysis (PCA), which reveals the main axes of variation and denoises the data.

In [ ]:
sc.tl.pca(adata, svd_solver='arpack', n_comps=150)

We can make a scatter plot in the PCA coordinates, but we will not use that later on.

Let us inspect the contribution of single PCs to the total variance in the data. This gives us information about how many PCs we should consider in order to compute the neighborhood relations of cells, e.g. used in the clustering function  `sc.tl.louvain()` or tSNE `sc.tl.tsne()`. In our experience, often a rough estimate of the number of PCs does fine.

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=150)

Save the result.

In [ ]:
adata.write(results_file, compression='gzip')

In [ ]:
adata

## Computing the neighborhood graph

Let us compute the neighborhood graph of cells using the PCA representation of the data matrix. You might simply use default values here. For the sake of reproducing Seurat's results, let's take the following values.

In [ ]:
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=70)

## Embedding the neighborhood graph

We suggest embedding the graph in two dimensions using UMAP ([McInnes et al., 2018](https://arxiv.org/abs/1802.03426)), see below. It is potentially more faithful to the global connectivity of the manifold than tSNE, i.e., it better preserves trajectories. In some ocassions, you might still observe disconnected clusters and similar connectivity violations. They can usually be remedied by running:

```
tl.paga(adata)
pl.paga(adata, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
tl.umap(adata, init_pos='paga')
```

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=['Classification'])

In [ ]:
sc.pl.umap(adata, color=['Epcam','Ptprc','Vim', 'Cdh1','Cdh2']) #'Ptprc',

In [ ]:
sc.pl.umap(adata, color=['Kras','Trp53'], vmax=5)
sc.pl.umap(adata, color=['mKate2','GFP'], vmax=2)

In [ ]:
sc.pl.umap(adata, color=['Hopx','Pdpn'], vmax=5)

In [ ]:
sc.pl.umap(adata, color=['total_counts'])

In [ ]:
# Score Cluster 5 cells
import csv
clusters = {}
clusterkeys = []
HPCS = False

def resetClusters(hpcs = 'cell2020'):
    clusters = {}
    clusterkeys = []
    HPCS = False
    try:
        if hpcs == 'tumorClusters':
            with open('clusters/tumorClustersv2.csv',encoding='utf-8-sig') as csvfile:
                csvreader = csv.reader(csvfile, delimiter=",")
                for row in csvreader:
                    clusters[row[0]] = row[1:]
            for i in range(0,16):
                clusterkeys.append('%i' % i)
            HPCS = '3'
        elif hpcs == 'cell2020':         
            with open('clusters/clusters_cell2020.csv',encoding='utf-8-sig') as csvfile:
                csvreader = csv.reader(csvfile, delimiter=",")
                for row in csvreader:
                    clusters[row[0]] = [x for x in row[1:] if x != '']
            for i in range(1,13):
                #if i == 9: continue
                clusterkeys.append('Cluster %i' % i)
            HPCS = 'Cluster 5'
        elif hpcs == 'cl5bootstrap':
            with open('clusters/cl5bootstrap.csv',encoding='utf-8-sig') as csvfile:
                csvreader = csv.reader(csvfile, delimiter=",")
                for row in csvreader:
                    clusters[row[0]] = row[1:]
            #for i in range(0,1):
                #clusterkeys.append('%i' % i)
            clusterkeys.append('Cl5Bootstrap')
            HPCS = 'Cl5Bootstrap'
        elif hpcs == 'cl5bootstrap2':
            with open('clusters/cl5bootstrap2.csv',encoding='utf-8-sig') as csvfile:
                csvreader = csv.reader(csvfile, delimiter=",")
                for row in csvreader:
                    clusters[row[0]] = row[1:]
            #for i in range(0,1):
                #clusterkeys.append('%i' % i)
            clusterkeys.append('Cl5Bootstrap2')
            HPCS = 'Cl5Bootstrap2'
        elif hpcs == 'shRenilla':
            raise ValueError
            ###
        else:
            raise ValueError
    except ValueError:
        print("%s is an invalid choice" % hpcs)
        raise
    return (clusters, clusterkeys, HPCS)

(clusters, clusterkeys, HPCS) = resetClusters('cell2020')

In [ ]:
def scoreAndPlot(ad, excludeList = None, groupby="Classification",rotation=90,numgenes=25,ctlgenes=25):
    #cmap = 'Reds' #colormap
    cmap = 'jet' #colormap
    if excludeList == None:
        for i in clusterkeys:
            if (numgenes > ctlgenes):
                ctlgenes = numgenes
            sc.tl.score_genes(ad, clusters[i][0:numgenes],score_name="%s" % i, ctrl_size=ctlgenes)  
        #sc.tl.score_genes(ad, clusters['Highly_mixed'],score_name="Highly_mixed")
  
        sc.pl.umap(ad, color=clusterkeys, color_map=cmap)
        #sc.pl.violin(ad, ['SLC4A11','ITGA2','CLDN4','TGIF1','Cluster5'],
         #   jitter=0.4, multi_panel=True, groupby=groupby, rotation=rotation)     
        #sc.pl.violin(ad, ['Highly_mixed'],
        #    jitter=0.4, multi_panel=True, groupby=groupby, rotation=rotation)            
        #sc.pl.violin(ad, ['MYC_sig','MYC','Fredlund','Jung'],
        #    jitter=0.4, multi_panel=True, groupby=groupby, rotation=rotation)     
        #sc.pl.violin(ad, ['ONECUT2','ONECUT2_369_sig','ONECUT2_29_sig'],
        #    jitter=0.4, multi_panel=True, groupby=groupby, rotation=rotation) 
        #sc.pl.violin(ad, ['YAP1', 'TAZ', 'YAP_sig'],
        #    jitter=0.4, multi_panel=True, groupby=groupby, rotation=rotation) 
        sc.pl.dotplot(ad, clusterkeys, groupby=groupby, swap_axes=True)

In [ ]:
from scipy.stats import ranksums

def HPCSViolinPlot(ad, cluster='0', groupby='leiden', score='Cluster5', save=None, singleGene=False):
    ad.obs['Cl5'] = 'not HPCS'
    ad.obs.loc[ad.obs[groupby].isin([cluster]), 'Cl5'] = 'HPCS'

    if singleGene:
        pvalue = ranksums(ad[ad.obs['Cl5'].isin(['HPCS'])][:,score].X.toarray(),ad[~ad.obs['Cl5'].isin(['HPCS'])][:,score].X.toarray())[1]
        #sc.pl.violin(ad, score, groupby='Cl5', xlabel = 'p = ' + str(pvalue), save=save)
    else:
        pvalue = ranksums(ad[ad.obs['Cl5'].isin(['HPCS'])].obs[score],ad[ad.obs['Cl5'].isin(['not HPCS'])].obs[score])[1]
    sc.pl.violin(ad, score, groupby='Cl5', xlabel = 'p = ' + str(pvalue), save=save)

In [ ]:
scoreAndPlot(adata, numgenes=500)

## Clustering the neighborhood graph

As with Seurat and many other frameworks, we recommend the Leiden graph-clustering method (community detection based on optimizing modularity) by [Traag *et al.* (2018)](https://scanpy.readthedocs.io/en/latest/references.html#traag18). Note that Leiden clustering directly clusters the neighborhood graph of cells, which we already computed in the previous section.

In [ ]:
sc.tl.leiden(adata, resolution=0.3)
#sc.tl.leiden(adata, resolution=0.25)

Plot the clusters, which agree quite well with the result of Seurat.

In [ ]:
sc.pl.umap(adata, color=['leiden', 'Epcam', 'Sftpc', # Tumor cells
                         'Ptprc']), #immune
                         #  'Cd4', 'Cd8a', #T cell #'Cd3e',
                         # 'Cd19', 'Ms4a1', 'Cd22', # B cell
                         # #'Itgax', 'Il3ra', # Dendritic Cell
                         # 'Ncam1',# 'Ncr1', # NK Cell
                         # 'Cd34', # Hematopoetic stem cell
                         # #'Cd14', 'Cd33', # macrophage
                         # #'Ceacam8' # Granulocyte
                         # 'Itga2b', 'Itgb3', 'Selp', #platelet
                         # #'Gypa',  # erythrocyte
                         # #'Mcam', 'Vcam1', 'Pecam1', 'Sele', # endothelial cell
                         # 'Cd109', 'Wnt5a', 'Kras'])

In [ ]:
sc.pl.umap(adata, color=['Kras',  'Zeb2', ]) #'Cd109', #'Sox11'

Save the result.

In [ ]:
sc.pl.umap(adata, color=['Slc4a11', 'Itga2', 'Cldn4','Epcam'], vmax=2) #'Tigit', 

In [ ]:
sc.pl.umap(adata, color=['Vim', 'Epcam', 'Krt8', 'Krt18', 'Pdpn', 'Hopx'], vmax=2) #'Pecam1',

In [ ]:
sc.pl.umap(adata, color=['GFP'], cmap='Reds')
sc.pl.umap(adata, color=['mKate2'], cmap='Reds')

In [ ]:
sc.pl.umap(adata, color=["shRenilla", "2118-Nfkb1", "2117-Nfkb1"], cmap='Reds')

In [ ]:
for label in adata.obs['Classification'].unique():
    sc.pl.umap(adata, color='Classification',groups=[label])

Cluster 1 appears to be fibroblasts based on Krt8 and Krt18 weak staining

In [ ]:
sc.pl.umap(adata,color=['n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'log10GenesPerUMI'])

In [ ]:
sc.pl.umap(adata, color=['leiden','mKate2'],cmap="Reds")

In [ ]:
sc.pl.umap(adata, color=['leiden'], groups=['0','1','2','5','6','7','10'])

## End Analysis

In [ ]:
# Select only those subset of cells that are useful
adata2 = sc.read('write/allmice.h5ad')
adata = adata2[adata.obs['leiden'].isin(['0','1','2','5','6','7','10']),:] # keep tumor cells
del adata2

In [ ]:
#fix a bug; 'base' is None but that doesn't seem to have transferred
adata.uns['log1p'] = {}
adata.uns['log1p']['base'] = None

In [ ]:
sc.pp.highly_variable_genes(adata)#, batch_key='Classification')

In [ ]:
#Ensure marker genes aren't making an impact to highly variable gene analysis downstream
adata.var.highly_variable['GFP'] = False
adata.var.highly_variable['mKate2'] = False
adata.var.highly_variable['Cre'] = False
for x in library:
    adata.var.highly_variable[x] = False

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata.raw = adata

In [ ]:
adata.write('write/allmice-tumor.h5ad', compression='gzip')
adata = sc.read('write/allmice-tumor.h5ad')

## Now rerun algorithm

In [ ]:
adata = adata[:, adata.var.highly_variable]

In [ ]:
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

In [ ]:
sc.pp.scale(adata, max_value=10)

## Principal component analysis

Reduce the dimensionality of the data by running principal component analysis (PCA), which reveals the main axes of variation and denoises the data.

In [ ]:
sc.tl.pca(adata, svd_solver='arpack', n_comps=150)

We can make a scatter plot in the PCA coordinates, but we will not use that later on.

In [ ]:
sc.pl.pca(adata, color='Cst3')

Let us inspect the contribution of single PCs to the total variance in the data. This gives us information about how many PCs we should consider in order to compute the neighborhood relations of cells, e.g. used in the clustering function  `sc.tl.louvain()` or tSNE `sc.tl.tsne()`. In our experience, often a rough estimate of the number of PCs does fine.

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=150)

Save the result.

In [ ]:
adata.write(results_file, compression='gzip')

In [ ]:
adata

## Computing the neighborhood graph

Let us compute the neighborhood graph of cells using the PCA representation of the data matrix. You might simply use default values here. For the sake of reproducing Seurat's results, let's take the following values.

In [ ]:
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=50)

## Embedding the neighborhood graph

We suggest embedding the graph in two dimensions using UMAP ([McInnes et al., 2018](https://arxiv.org/abs/1802.03426)), see below. It is potentially more faithful to the global connectivity of the manifold than tSNE, i.e., it better preserves trajectories. In some ocassions, you might still observe disconnected clusters and similar connectivity violations. They can usually be remedied by running:

```
tl.paga(adata)
pl.paga(adata, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
tl.umap(adata, init_pos='paga')
```

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=['Classification'])
#sc.pl.umap(adata, color=['Porcn', 'Lgr5', 'Lgr6','Axin2']) #LGR5 not found #
sc.pl.umap(adata, color=['Epcam', 'Ptprc', 'Vim', 'Cdh1','Cdh2']) #CDH2 not found
sc.pl.umap(adata, color=['Slc4a11', 'Itga2', 'Cldn4']) #, 'Abcb1'
sc.pl.umap(adata, color=['Hopx', 'Pdpn'])
#sc.pl.umap(adata, color=['Cd81', 'Cd19'])
#sc.pl.umap(adata, color=['Trp53','Meg3'], vmax=5)

In [ ]:
sc.pl.umap(adata, color=['Vim', 'Onecut2', 'mKate2', 'GFP']) #'Sox11', 

In [ ]:
#NFkB targets
nfkb = ['Tnf', 'Mmp13','Nfkbia']
nfkball = ['Tnf', 'Mmp13', 'Nfkbia', 'Bcl2', 'Bcl2l1', 'Xiap', 'Birc2', 'Il6', 'Birc5', 'Ccnd1', 'Mmp3']
nfkbfactors = ['Nfkb1', 'Nfkb2', 'Rela', 'Rel', 'Relb']
sc.pl.umap(adata,color=nfkb)
sc.tl.score_genes(adata, nfkball)
sc.pl.umap(adata, color=nfkbfactors)

In [ ]:
sc.pl.umap(adata, color='Slc4a11')

In [ ]:
(clusters, clusterkeys, HPCS) = resetClusters('cell2020')
scoreAndPlot(adata, numgenes=500)

In [ ]:
(clusters, clusterkeys, HPCS) = resetClusters('cl5bootstrap2')
scoreAndPlot(adata,numgenes=100)

In [ ]:
sc.pl.umap(adata, color=['GFP'], cmap='Reds')
sc.pl.umap(adata, color=['mKate2'], cmap='Reds')
sc.pl.umap(adata, color=['Nfkb1'], cmap='Reds')
sc.pl.umap(adata, color=['Nfkb1', "2117-Nfkb1","2118-Nfkb1",'shRenilla'], cmap='Reds')
sc.pl.umap(adata, color=["2118-Nfkb1"], cmap='Reds')

In [ ]:
sc.pl.umap(adata, color=['n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'log10GenesPerUMI'])

In [ ]:
sc.pl.umap(adata,color='Cluster 5', cmap='Reds')

In [ ]:
sc.pl.umap(adata,color='Cluster 11', cmap='Reds')

In [ ]:
sc.tl.embedding_density(adata, basis='umap', groupby='Classification')

In [ ]:
sc.pl.embedding_density(adata, basis='umap', key='umap_density_Classification')

In [ ]:
sc.pl.matrixplot(adata, ['Nfkb1','Slc4a11','Cluster 5'], groupby='Classification', standard_scale='var')

In [ ]:
adata.obs.Classification.value_counts()

## Clustering the neighborhood graph

As with Seurat and many other frameworks, we recommend the Leiden graph-clustering method (community detection based on optimizing modularity) by [Traag *et al.* (2018)](https://scanpy.readthedocs.io/en/latest/references.html#traag18). Note that Leiden clustering directly clusters the neighborhood graph of cells, which we already computed in the previous section.

In [ ]:
sc.tl.leiden(adata, resolution=.275)
sc.pl.umap(adata, color=['leiden','Slc4a11','Cluster 5'])

Plot the clusters, which agree quite well with the result of Seurat.

In [ ]:
import matplotlib.pyplot as plt
_ = plt.hist(adata.obs['Cluster 5'], bins='auto')
plt.ylabel("Count")
plt.title("Cluster 5 Score")
plt.axvline(x=0.2)
plt.show()

In [ ]:
# import libraries (some are for cosmetics)
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from scipy.stats import norm
from sklearn.mixture import BayesianGaussianMixture
import matplotlib as mpl

x = np.array(adata.obs['Cluster 5']).reshape(-1, 1)

# create GMM model object
gmm = BayesianGaussianMixture(n_components=2, random_state=250,init_params='k-means++')

# find useful parameters
fitGMM = gmm.fit(x)
mean = fitGMM.means_  
covs  = fitGMM.covariances_
weights = fitGMM.weights_

# create necessary things to plot
x_axis = np.arange(0, 1, 0.01)
y_axis0 = norm.pdf(x_axis, float(mean[0][0]), np.sqrt(float(covs[0][0][0])))*weights[0] # 1st gaussian
y_axis1 = norm.pdf(x_axis, float(mean[1][0]), np.sqrt(float(covs[1][0][0])))*weights[1] # 2nd gaussian
cutoff = 0
if mean[0][0] < mean[1][0]:
    cutoff = x_axis[np.where(y_axis1 < y_axis0)[0][-1]+2]
else:
    cutoff = x_axis[np.where(y_axis0 < y_axis1)[0][-1]+2]
    

ax = fig.add_subplot(1,2,2)
# Plot 2
plt.hist(x, density=True, color='black', bins=100)
plt.plot(x_axis, y_axis0, lw=3, c='C6')
plt.plot(x_axis, y_axis1, lw=3, c='C1')
plt.plot(x_axis, y_axis0+y_axis1, lw=3, c='C0', ls=':')
#plt.xlim(0, 4, 0.1)
#plt.ylim(0.0, 2.0)
plt.xlabel(r"Xlog10 + 1 counts", fontsize=20)
plt.ylabel(r"freq", fontsize=20)
plt.axvline(x=cutoff)

#plt.subplots_adjust(wspace=0.3)
plt.show()
plt.close('all')
print(fitGMM.converged_)

In [ ]:
adata.obs['Cluster 5 high'] = ['1' if z >cutoff else '-1' for z in adata.obs['Cluster 5']]


sc.pl.umap(adata, color=['Cluster 5 high'], cmap="tab20c_r")

In [ ]:
#fix a bug; 'base' is None but that doesn't seem to have transferred
adata.uns['log1p'] = {}
adata.uns['log1p']['base'] = None

Save the result.

In [ ]:
sc.pl.umap(adata, color=['leiden', 'Cluster 5', 'Cluster 11'])

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from joblib import dump, load
names = load('genes.joblib')

In [ ]:
adata_log = adata[:,adata.var.index.isin(names)]

In [ ]:
adata_names = [x for x in adata_log.var.index]

In [ ]:
goi = [x for x in adata_log.var.index]

## Load SmartSeq h5ad file

In [ ]:
smartseq = anndata.read('smartseq.h5ad')

## Logistic Regression against Cancer Cell 2020 Clusters intersected with 10x data

In [ ]:
from sklearn.model_selection import train_test_split

names_adata = smartseq[:,smartseq.var.index.isin(goi)]

X_train, X_test, y_train, y_test = train_test_split(names_adata.X, smartseq.obs['clusterK12'], test_size=0.25, random_state=250)

In [ ]:
names = [x for x in names_adata.var.index]

In [ ]:
# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
#logreg = LogisticRegression(random_state=250)
#logreg = LogisticRegression(random_state=250,multi_class='multinomial',solver='saga')
logreg = LogisticRegression(random_state=250,multi_class='multinomial', solver='lbfgs')

# fit the model with data
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
y_score = logreg.predict_proba(X_test)


In [ ]:
# import the metrics class
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
# import required modules
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

#Text(0.5,257.44,'Predicted label');

In [ ]:
from sklearn.metrics import classification_report
target_names = ['1', '2', '3','4','5','6','7','8','9','10','11','12']
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_onehot_test.shape  # (n_samples, n_classes)

In [ ]:
class_of_interest = 9
class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
class_id

In [ ]:
import matplotlib.pyplot as plt

from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_predictions(
    y_onehot_test[:, class_id],
    y_score[:, class_id],
    name=f"{class_of_interest} vs the rest",
    color="darkorange",
    #plot_chance_level=True,
)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves:\nVirginica vs (Setosa & Versicolor)")
plt.legend()
plt.show()

In [ ]:
## Now write logit classifier to load later
from joblib import dump, load
dump(logreg, 'cancerCell2020Logreg_intersecting-all.joblib')
dump(names, 'intersecting_genes_new-all.joblib')

In [ ]:
y_pred = logreg.predict(smartseq[:,names].X)
y_proba = logreg.predict_proba(smartseq[:,names].X)

In [ ]:
smartseq.obs['clusterK12_intersecting'] = ["%02d" % x for x in y_pred]
smartseq.obs['clusterK12_intersecting'] = smartseq.obs['clusterK12_intersecting'].astype('category')

In [ ]:
#results_binary = y_pred
#probs_binary = y_proba

results_binary = logreg.predict(X_test)
probs_binary = logreg.predict_proba(X_test)

results_df_binary = pd.DataFrame()
results_df_binary['predicted'] = list(results_binary)
#results_df_binary['actual'] = list(smartseq.obs['clusterK12'])
results_df_binary['actual'] = list(y_test)
for x in range(0,12):
    results_df_binary["probability_%d" % (x+1)] = [i[x] for i in list(probs_binary)]
results_df_binary['highest_prob'] =results_df_binary[["probability_%d" % (x+1) for x in range(0,12)]].max(axis=1)
results_df_binary['correct'] = (results_df_binary['predicted'] == results_df_binary['actual']).astype(int)
results_df_binary

In [ ]:
results_df_binary['bin'] = pd.qcut(results_df_binary['highest_prob'], q=7)
idx = pd.IntervalIndex(results_df_binary['bin'])
results_df_binary['low'] = idx.left
results_df_binary['high'] = idx.right
results_df_binary['prob range'] = results_df_binary['low'].round(2).astype(str) + "-" + results_df_binary['high'].round(2).astype(str)
df_bins_grouped = results_df_binary[['low','high','correct','prob range']].groupby('prob range').mean().reset_index()
df_bins_grouped

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(4,4))
plt.subplots_adjust(top=0.85)
#plt.figure(figsize=(11,9))
#plt.subplots_adjust(top=0.85)

ax = sns.barplot(x='prob range', y='correct', data=df_bins_grouped)
plt.title("Logistic Regression Accuracy by Probability Range")
plt.ylabel('Percent correct')
plt.xlabel('Probability range')
plt.show()

In [ ]:
#reset size
from matplotlib import rcParams
FIGSIZE=(4,4)
rcParams['figure.figsize']=FIGSIZE

In [ ]:
sc.pl.umap(smartseq, color=['clusterK12','clusterK12_intersecting'])

In [ ]:
dict = {1: "01", 2: "02", 3: "03", 4: "04", 5: "05", 6: "06", 7: "07", 8: "08", 9: "09", 10: "10", 11: "11", 12: "12"}

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(smartseq.obs.replace({"clusterK12": dict})['clusterK12'], smartseq.obs['clusterK12_intersecting'])

In [ ]:
results_binary = logreg.predict(smartseq[:,names].X)
probs_binary = logreg.predict_proba(smartseq[:,names].X)

results_df_binary = pd.DataFrame()
results_df_binary['predicted'] = list(results_binary)
#results_df_binary['actual'] = list(smartseq.obs['clusterK12'])
for x in range(0,12):
    results_df_binary[x+1] = [i[x] for i in list(probs_binary)]
results_df_binary['highest_prob'] =results_df_binary[[x+1 for x in range(0,12)]].max(axis=1)
results_df_binary

In [ ]:
_ = plt.hist(results_df_binary['highest_prob'], bins='auto', histtype='step')

## End LogisticRegression

In [ ]:
adata_ordered_genes = [adata_names.index(x) for x in names]

In [ ]:
adata_log_ordered = adata_log[:,adata_ordered_genes]

In [ ]:
y_pred = logreg.predict(adata_log_ordered.X)

In [ ]:
adata_log_ordered.obs['clusterK12'] = ["%02d" % x for x in y_pred]
adata_log_ordered.obs['clusterK12'] = adata_log_ordered.obs['clusterK12'].astype('category')

In [ ]:
#reset size
from matplotlib import rcParams
FIGSIZE=(4,4)
rcParams['figure.figsize']=FIGSIZE

In [ ]:
sc.pl.umap(adata_log_ordered,color=['leiden','clusterK12'], wspace=0.1)

In [ ]:
sc.pl.umap(adata_log_ordered,color=['clusterK12'])

In [ ]:
for x in adata_log_ordered.obs.clusterK12.cat.categories:
    sc.pl.umap(adata_log_ordered,color=['clusterK12'], groups=[x])

In [ ]:
# bug fix
adata_log_ordered.uns['log1p']['base'] = None

In [ ]:
 cell_proportion_df = adata_log_ordered.obs.leiden.value_counts().T.plot(kind='bar', stacked=True, legend=False)

In [ ]:
 cell_proportion_df = adata_log_ordered.obs.clusterK12.value_counts(sort=False).T.plot(kind='bar', stacked=True, legend=False)

In [ ]:
 cell_proportion_df = pd.crosstab(adata_log_ordered.obs['leiden'],adata_log_ordered.obs['clusterK12'], normalize='columns').T.plot(kind='bar', stacked=True).legend(bbox_to_anchor=(1,1))

In [ ]:
results_binary = logreg.predict(adata_log_ordered.X)
probs_binary = logreg.predict_proba(adata_log_ordered.X)

results_df_binary = pd.DataFrame()
results_df_binary['predicted'] = list(results_binary)
#results_df_binary['actual'] = list(smartseq.obs['clusterK12'])
for x in range(0,12):
    results_df_binary[x+1] = [i[x] for i in list(probs_binary)]
results_df_binary['highest_prob'] =results_df_binary[[x+1 for x in range(0,12)]].max(axis=1)
results_df_binary

In [ ]:
_ = plt.hist(results_df_binary['highest_prob'], bins='auto')
plt.axvline(x=0.9)

In [ ]:
results_df_binary['bin'] = pd.qcut(results_df_binary['highest_prob'], q=7)
idx = pd.IntervalIndex(results_df_binary['bin'])
results_df_binary['low'] = idx.left
results_df_binary['high'] = idx.right
results_df_binary['prob range'] = results_df_binary['low'].round(2).astype(str) + "-" + results_df_binary['high'].round(2).astype(str)
df_bins_grouped = results_df_binary[['low','high','prob range']].groupby('prob range').mean().reset_index()
df_bins_grouped

In [ ]:
cutoff = 0.8
adata_log_ordered.obs['clusterK12_stringent'] = 'other'
adata_log_ordered.obs['clusterK12_stringent'][np.array(results_df_binary['highest_prob'] >= cutoff)] = ["%02d" % y for y in results_df_binary[[x+1 for x in range(0,12)]].idxmax(axis=1)[np.array(results_df_binary['highest_prob'] >= cutoff)].to_numpy()]

In [ ]:
adata_log_ordered.obs['clusterK12_stringent']

In [ ]:
adata_log_ordered.obs['clusterK12_stringent'] = adata_log_ordered.obs['clusterK12_stringent'].astype('category')

In [ ]:
sc.pl.umap(adata_log_ordered, color=['clusterK12', 'clusterK12_stringent'], wspace=0.5)

In [ ]:
 cell_proportion_df = pd.crosstab(adata_log_ordered.obs['clusterK12_stringent'],adata_log_ordered.obs['clusterK12'],normalize='columns').T.plot(kind='bar', stacked=True).legend(bbox_to_anchor=(1,1))

In [ ]:
 cell_proportion_df = pd.crosstab(adata_log_ordered.obs['leiden'],adata_log_ordered.obs['clusterK12_stringent'], normalize='columns').T.plot(kind='bar', stacked=True).legend(bbox_to_anchor=(1,1))

In [ ]:
 cell_proportion_df = pd.crosstab(adata_log_ordered.obs['Group'],adata_log_ordered.obs['clusterK12_stringent'], normalize='columns').T.plot(kind='bar', stacked=True).legend(bbox_to_anchor=(1,1))

In [ ]:
cancerCell = ['Cluster 1', 'Cluster 2','Cluster 3','Cluster 4','Cluster 5','Cluster 6', 'Cluster 7', 
              'Cluster 8', 'Cluster 9', 'Cluster 10', 'Cluster 11', 'Cluster 12']
#cell_proportion_df = pd.crosstab(adata_log_ordered.obs['Cluster 1'],adata_log_ordered.obs['clusterK12']).T.plot(kind='bar', stacked=True).legend(bbox_to_anchor=(1,1))
sc.pl.matrixplot(adata_log_ordered, cancerCell, 'clusterK12', standard_scale='var')
sc.pl.matrixplot(adata_log_ordered, cancerCell, 'clusterK12_stringent', standard_scale='var')

In [ ]:
for x in adata_log_ordered.obs.clusterK12_stringent.cat.categories:
    # skip if NA
    sc.pl.umap(adata_log_ordered, color='clusterK12_stringent', groups=[x])

In [ ]:
sc.tl.rank_genes_groups(adata_log_ordered, 'clusterK12', method='logreg')
sc.pl.rank_genes_groups(adata_log_ordered, n_genes=25, sharey=False)

In [ ]:
adata_log_ordered.obs.clusterK12.value_counts()

In [ ]:
cancerCell = ['Cluster 1', 'Cluster 2','Cluster 3','Cluster 4','Cluster 5','Cluster 6', 'Cluster 7', 
              'Cluster 8', 'Cluster 9', 'Cluster 10', 'Cluster 11', 'Cluster 12']
#sc.pl.violin(adata_log_ordered, cancerCell, groupby='clusterK12')

In [ ]:
sc.pl.stacked_violin(adata_log_ordered, cancerCell, groupby='clusterK12_stringent')

In [ ]:
sc.pl.dotplot(adata_log_ordered, cancerCell, groupby='clusterK12_stringent')

In [ ]:
sc.pl.matrixplot(adata_log_ordered, cancerCell, groupby='Group',standard_scale='var')

In [ ]:
sc.pl.matrixplot(adata_log_ordered, ['Nfkb1','Nfkbia'], groupby='Group',standard_scale='var')

In [ ]:
adata.obs['clusterK12'] = adata_log_ordered.obs['clusterK12'].astype('category')
adata.obs['clusterK12_stringent'] = adata_log_ordered.obs['clusterK12_stringent'].astype('category')

In [ ]:
 cell_proportion_df = pd.crosstab(adata.obs['leiden'],adata_log_ordered.obs['clusterK12_stringent'], normalize='columns').T.plot(kind='bar', stacked=True).legend(bbox_to_anchor=(1,1))

In [ ]:
 cell_proportion_df = pd.crosstab(adata.obs['Group'],adata_log_ordered.obs['clusterK12_stringent'], normalize='columns').T.plot(kind='bar', stacked=True).legend(bbox_to_anchor=(1,1))

## Now retrain a new Logit classifier based off the single cell 10x data

## Logistic Regression against Cancer Cell 2020 Clusters in 10x data (stringent)

In [ ]:
adata.obs.Group.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

names_adata = adata[:,~adata.var.index.isin(['mKate2','Cre','GFP'])]
names_adata = names_adata[names_adata.obs.Group.isin(['Renilla_Control']),:]

X_train, X_test, y_train, y_test = train_test_split(names_adata[~(names_adata.obs['clusterK12_stringent'] == 'other'),:].X, names_adata[~(names_adata.obs['clusterK12_stringent'] == 'other'),:].obs['clusterK12'], test_size=0.25, random_state=250)

In [ ]:
names = [x for x in names_adata.var.index]

In [ ]:
# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=250, multi_class='multinomial', solver='saga')

# fit the model with data
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
y_score = logreg.predict_proba(X_test)


In [ ]:
pd.DataFrame(y_pred).value_counts()

In [ ]:
# import the metrics class
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
# import required modules
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

#Text(0.5,257.44,'Predicted label');

In [ ]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_onehot_test.shape  # (n_samples, n_classes)

In [ ]:
label_binarizer.classes_

In [ ]:
class_of_interest = '05'
class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
class_id

In [ ]:
import matplotlib.pyplot as plt

from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_predictions(
    y_onehot_test[:, class_id],
    y_score[:, class_id],
    name=f"{class_of_interest} vs the rest",
    color="darkorange",
    #plot_chance_level=True,
)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves:\nVirginica vs (Setosa & Versicolor)")
plt.legend()
plt.show()

In [ ]:
## Now write logit classifier to load later
from joblib import dump, load
dump(logreg, 'cancerCell2020Logreg_retrain-all.joblib')
dump(names, 'genes_retrain-all.joblib')

In [ ]:
y_pred = logreg.predict(adata.X)

In [ ]:
adata.obs['clusterK12_retrain'] = y_pred

In [ ]:
adata.obs['clusterK12_retrain'] = adata.obs['clusterK12_retrain'].astype('category')

In [ ]:
sc.pl.umap(adata, color = ['clusterK12','clusterK12_retrain'], wspace=0.5)

In [ ]:
 cell_proportion_df = adata.obs.clusterK12.value_counts(sort=False).T.plot(kind='bar', stacked=True, legend=False)

In [ ]:
 cell_proportion_df = adata.obs.clusterK12_retrain.value_counts(sort=False).T.plot(kind='bar', stacked=True, legend=False)

In [ ]:
 cell_proportion_df = pd.crosstab(adata.obs['clusterK12_retrain'],adata.obs['clusterK12'], normalize='columns').T.plot(kind='bar', stacked=True).legend(bbox_to_anchor=(1,1))

In [ ]:
sc.pl.stacked_violin(adata, cancerCell, 'clusterK12')

In [ ]:
sc.pl.stacked_violin(adata, cancerCell, 'clusterK12_retrain')

## End Logistic Regression

In [ ]:
sc.pl.umap(adata, color=['Onecut2'])

In [ ]:
sc.pl.umap(adata, color=['Slc4a11','Cldn4','Itga2', 'Plaur']) #'Tigit',

In [ ]:
sc.pl.umap(adata, color=['Plaur','Itgb1','Cd151', 'Cd9'], cmap="jet")

In [ ]:
sc.pl.umap(adata, color=['Vim',], cmap="jet") #'Cd109'

In [ ]:
sc.pl.umap(adata, color=['Hopx','Pdpn'], cmap="jet")

In [ ]:
sc.pl.umap(adata, color=['Smad2', 'Smad3'], cmap="jet")

In [ ]:
adata.write_h5ad(results_file)

In [ ]:
for label in adata.obs['Classification'].unique():
    sc.pl.umap(adata, color='Classification',groups=[label])

In [ ]:
samples

In [ ]:
sc.tl.embedding_density(adata,basis='umap', groupby='Group')

In [ ]:
adata.obs.Group.value_counts()

In [ ]:
sc.pl.embedding_density(adata,basis='umap',key='umap_density_Group', 
                        group=['Renilla_Control','2117_Control','2118_Control',
                              'Renilla_EGFRi','2117_EGFRi','2118_EGFRi'])

In [ ]:
sc.tl.embedding_density(adata,basis='umap', groupby='clusterK12')

In [ ]:
sc.pl.embedding_density(adata,basis='umap',key='umap_density_clusterK12')

In [ ]:
adata.obs[['Group', 'clusterK12']].value_counts()

In [ ]:
sc.pl.violin(adata, ['Nfkb1','Rel','Rela','Nfkb2','Relb','Cluster 5','Cl5Bootstrap2'], groupby='Group',rotation=90,inner='box',stripplot=False)

In [ ]:
sc.pl.dotplot(adata, ['Nfkb1','Rel','Rela','Nfkb2','Relb','Nfkbia','Tnf','Il1r1'], groupby='Group') #'Il1b'

In [ ]:
sc.pl.violin(adata, ['Nfkbia'],groupby='Group', rotation=90, stripplot=False, inner='box')

In [ ]:
sc.pl.matrixplot(adata, ['Nfkb1','Nfkbia','Slc4a11','Itga2','Cluster 5','Cl5Bootstrap2'], groupby='Group', standard_scale='var')

In [ ]:
sc.pl.dotplot(adata, ['Nfkb1','Nfkbia','Slc4a11','Itga2','Cluster 5'], groupby='Group', standard_scale='var')

In [ ]:
sc.pl.dotplot(adata, ['Nfkb1','Nfkbia','Slc4a11','Itga2','Cluster 5'], groupby='Group', standard_scale='var')

In [ ]:
adata.obs.Group.value_counts()